This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([1.50371026e+02, 3.25497888e+02, 2.10375206e+02, 2.76217369e+02,
       2.19470531e+02, 3.46912901e+02, 1.88139584e+02, 1.20565464e+02,
       1.20156284e+02, 9.62792101e+01, 2.00047469e+02, 1.76193353e+02,
       8.53211606e+01, 1.30710796e+02, 1.70112526e+02, 1.72420203e+02,
       1.71679718e+02, 1.66699389e+02, 1.58791749e+02, 1.78669886e+02,
       1.87709915e+02, 1.79995444e+02, 1.25772961e+02, 3.69780134e+02,
       3.35551879e+02, 2.27914069e+02, 5.49497902e+01, 1.19298374e+02,
       1.03964994e+02, 1.16890895e+02, 7.70450205e+01, 9.11569365e+01,
       8.38092566e+01, 2.28914531e+02, 1.68464956e+02, 4.50731986e+01,
       9.24493164e+01, 1.63185749e+02, 3.90426011e+02, 1.21368422e+02,
       8.39207969e+01, 1.08666855e+02, 1.29128414e+02, 2.91384616e+02,
       4.08488586e+02, 7.39283454e+01, 1.22389663e+02, 1.32814981e+02,
       6.86792014e+01, 6.71398750e+01, 8.64303692e+01, 1.75379626e+02,
       2.37521275e+02, 1.63297865e+02, 3.47383100e+02, 6.94361418e+01,
       5.73220058e+01, 2.94591088e+01, 0.00000000e+00, 1.56627930e+02,
       9.79676617e+00, 1.58218632e+02, 5.61529988e+01, 3.63631922e+01,
       5.42920320e+01, 0.00000000e+00, 6.45049606e+01, 0.00000000e+00,
       6.46627457e+01, 4.00299318e+01, 2.68896498e+01, 3.07830539e+02,
       0.00000000e+00, 1.40134457e+02, 8.03088872e+01, 7.93342932e+01,
       7.03541024e+01, 1.39599684e+02, 0.00000000e+00, 1.27825789e+02,
       1.49850095e+02, 1.19774983e+02, 5.00254523e+01, 0.00000000e+00,
       0.00000000e+00, 3.39191772e+01, 6.01231126e+01, 4.09146349e+01,
       0.00000000e+00, 0.00000000e+00, 2.03972667e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.70304392e+01, 0.00000000e+00, 0.00000000e+00, 3.20782685e+01,
       2.68282539e+01, 3.35683107e+01, 1.26328149e+01, 2.63184751e+01,
       0.00000000e+00, 6.03605982e+01, 7.74558848e+00, 2.51748038e+01,
       6.37138607e+00, 0.00000000e+00, 3.65075124e+01, 0.00000000e+00,
       1.92047597e+01, 1.78001455e+02, 1.89644510e+02, 1.31304680e+02,
       1.33559658e+02, 1.40198319e+02, 0.00000000e+00, 6.92693877e+01,
       0.00000000e+00, 0.00000000e+00, 6.53557069e+01, 2.08989075e+02,
       3.32000681e+01, 2.91224504e+01, 4.91573257e+01, 8.68896540e+01,
       1.57595600e+02, 1.37408713e+02, 1.50999876e+02, 1.07975067e+02,
       1.24630840e+02, 7.79804428e+01, 8.52336429e+01, 1.00949225e+02,
       0.00000000e+00, 8.77843631e+01, 1.93409720e+01, 0.00000000e+00,
       4.36717688e+00, 1.27378189e+02, 1.58427043e+02, 2.85948164e+02,
       9.97625282e+01, 2.80876470e+02, 1.65778433e+02, 1.75569398e+02,
       2.30921905e+02, 1.34506442e+02, 7.31341625e+01, 5.73822138e+01,
       2.50244873e+01, 6.49749186e+01, 1.31808017e+01, 2.62303388e+01,
       9.10665100e+00, 0.00000000e+00, 0.00000000e+00, 7.81817223e+01,
       5.54728596e+01, 5.25651706e+01, 4.71419734e+01, 4.83755926e+01,
       3.08078053e+01, 6.98113911e+01, 1.55465078e+02, 2.34800709e+02,
       7.83502295e+01, 9.13811396e+01, 7.45637353e+01, 6.94770317e+01,
       2.02393592e+02, 6.06316356e+01, 8.26646597e+01, 1.10052006e+02,
       4.07604224e+01, 3.92079232e+01, 1.69719765e+02, 2.24402165e+02,
       0.00000000e+00, 1.06701247e+02, 9.38711282e+01, 0.00000000e+00,
       9.13449975e+01, 0.00000000e+00, 1.20596615e+02, 6.59849632e+01,
       8.10394982e+01, 2.65092677e+01, 4.29040665e+01, 0.00000000e+00,
       2.87242187e+01, 4.72433761e+01, 2.18195920e+01, 6.67613331e+01,
       7.11848257e+01, 5.29102783e+01, 7.01397816e+01, 0.00000000e+00,
       9.40584552e+01, 8.58908577e+01, 8.06894184e+01, 1.25215370e+02,
       4.21875225e+01, 4.45467226e+01, 1.03743969e+02, 1.20413789e+02,
       1.91113049e+02, 1.68517827e+02, 2.86026589e+02, 3.78453371e+02,
       8.83272115e+01, 1.16180977e+02, 1.13869162e+02, 1.14407716e+02,
       1.14507152e+02, 1.11920471e+02, 1.14498162e+02, 1.22772579e+02,
       1.24314124e+02, 1.50672468e+02, 1.63799175e+02, 2.85344428e+02,
       1.69625113e+02, 1.91836674e+02, 1.23246629e+02, 8.70459947e+01,
       2.12302508e+02, 1.78586168e+02, 2.50579269e+02, 2.73613179e+02,
       2.23816699e+02, 3.40437655e+02, 2.13493682e+02, 2.33810236e+02,
       1.87894409e+02, 1.43517205e+02, 7.69784095e+01, 9.12847572e+01,
       2.12756631e+02, 8.35495499e+01, 1.61293351e+02, 2.08659157e+02,
       1.92088747e+02, 1.00117427e+02, 1.92712755e+02, 1.83449190e+02,
       1.02794584e+02, 1.85807040e+02, 2.16454613e+02, 2.53019864e+02,
       2.38522270e+02, 3.15514875e+02, 3.17205543e+02, 2.98525793e+02,
       3.28541723e+02, 3.46087908e+02, 1.64002238e+02, 8.54936441e+01,
       1.04459645e+02, 8.85969515e+01, 1.38575932e+02, 7.93248059e+01,
       8.95472627e+01, 2.13365657e+02, 1.36443088e+02, 1.47525298e+02,
       5.95241562e+01, 1.38091226e+02, 7.74665625e+01, 5.68526124e+01,
       1.43519281e+02, 6.92085003e+01, 1.99578096e+02, 2.10329787e+02,
       2.99112578e+02, 2.47909595e+02, 1.73746536e+02, 1.83572992e+02,
       2.04554894e+02, 8.57231665e+02, 1.05932372e+02, 8.42083837e+01,
       9.57786406e+01, 1.23367070e+02, 5.19883620e+01, 7.70898973e+01,
       1.04682468e+02, 1.43722366e+02, 1.90046405e+02, 1.05836151e+02,
       1.51089123e+02, 1.25309562e+02, 1.56518089e+02, 3.90465522e+01,
       1.17389412e+02, 1.13321950e+02, 7.09524931e+01, 3.26473248e+01,
       1.25171931e+02, 7.21745594e+01, 4.28892788e+01, 5.66837962e+01,
       9.42635786e+01, 3.27734114e+01, 4.29043523e+01, 1.61563184e+02,
       1.49540965e+02, 1.87848591e+02, 1.40218828e+02, 1.24017817e+02,
       1.74909069e+02, 2.11335266e+02, 1.78369049e+02, 2.10616630e+02,
       1.13527293e+02, 1.63793911e+02, 1.46552154e+02, 1.85543114e+02,
       2.20408702e+02, 1.63982208e+02, 5.49152363e+01, 1.08537384e+02,
       1.23935967e+02, 1.13886204e+02, 1.10285836e+02, 1.83271284e+02,
       1.04244208e+02, 1.37583441e+02, 1.64660391e+02, 9.87844184e+01,
       2.98245707e+02, 1.62419081e+02, 1.38582903e+02, 3.35321293e+02,
       8.86858956e+01, 8.28791006e+01, 2.97477073e+01, 4.41815882e+01,
       1.83475519e+02, 4.69172371e+01, 2.71330052e+01, 7.10907636e+01,
       2.23792303e+01, 8.71299483e+01, 9.53050436e+01, 7.29422616e+01,
       1.13170481e+02, 9.56649845e+01, 5.04471564e+01, 7.84670662e+01,
       4.06890701e+01, 3.50459396e+01, 1.05859697e+02, 1.43075475e+02,
       1.40737465e+02, 9.38114347e+01, 6.35864630e+01, 0.00000000e+00,
       1.61966268e+02, 1.32019981e+02, 1.78944559e+02, 3.87496682e+01,
       0.00000000e+00, 6.20947510e+01, 0.00000000e+00, 0.00000000e+00,
       6.41409237e+01, 2.41527621e+01, 4.54043655e+01, 1.00472922e+02,
       1.60334633e+02, 0.00000000e+00, 6.15578071e+01, 2.63539598e+01,
       3.38853640e+01, 9.31924847e+00, 2.15324072e+01, 0.00000000e+00,
       2.84070013e+01, 3.73769471e+01, 8.47289652e+01, 4.56348093e+01,
       7.48181406e+01, 2.71292986e+01, 0.00000000e+00, 2.99157066e+01,
       7.07659236e+01, 2.67038101e+01, 3.74680233e+01, 8.17268550e+01,
       5.51408019e+01, 1.16752352e+02, 7.05986967e+01, 8.22454137e+01,
       9.72311934e+01, 3.81925114e+01, 0.00000000e+00, 4.38219567e+01,
       4.23854981e+01, 4.08206874e+01, 3.91274706e+01, 5.66656498e+01,
       1.90870878e+02, 1.23849520e+02, 0.00000000e+00, 8.25826337e+01,
       7.56793957e+01, 7.09430276e+01, 9.01656399e+01, 8.48759224e+01,
       4.59371875e+01, 5.19044272e+01, 5.36862536e+01, 5.00275181e+01,
       7.80670028e+01, 1.73113038e+02, 3.30048005e+01, 0.00000000e+00,
       0.00000000e+00])
y_pred = array([ 1.58731058e+02,  2.63517532e+02,  2.06496850e+02,  1.70727805e+02,
        1.55872287e+02,  2.77691353e+02,  2.03325143e+02,  1.29034258e+02,
        8.92251701e+01,  9.09991402e+01,  1.53425014e+02,  1.67381233e+02,
        9.72930359e+01,  9.94873399e+01,  1.73562702e+02,  2.01906754e+02,
        1.74042354e+02,  1.95157240e+02,  1.59642617e+02,  1.70822319e+02,
        1.61731106e+02,  1.58582693e+02,  1.52635670e+02,  3.07570021e+02,
        2.61794894e+02,  2.35160490e+02,  1.02183685e+02,  1.71854773e+02,
        1.08995910e+02,  1.06956799e+02,  1.08096480e+02,  7.16234419e+01,
        8.79551986e+01,  2.25244168e+02,  1.94615132e+02,  5.29267266e+01,
        9.42015905e+01,  1.32565160e+02,  2.67525358e+02,  8.02084278e+01,
        7.16379683e+01,  1.04523745e+02,  8.56071271e+01,  1.79766828e+02,
        3.74903367e+02,  6.27683645e+01,  6.90232359e+01,  7.65220370e+01,
        8.22503057e+01,  6.41625146e+01,  9.70302337e+01,  1.50384526e+02,
        2.24773964e+02,  5.46227809e+01,  1.81423223e+02,  9.04583270e+01,
        8.28871352e+01,  1.02514377e+02,  1.32406820e+02,  6.50547494e+01,
        1.58587339e+02,  1.11919140e+02,  1.55635537e+02,  1.16915585e+02,
        6.23604117e+01,  9.11896580e+01,  1.12203563e+02,  1.80030654e+01,
        9.39783000e+01,  8.30972864e+01,  6.88384107e+01,  1.21082751e+02,
        3.05125466e+01,  1.60897057e+02,  4.26387237e+01,  6.37759393e+01,
        1.45793604e+02,  1.46523067e+02,  4.15291094e+01,  1.21465690e+02,
        1.29890076e+02,  1.42913379e+02,  7.39018738e+01,  8.24252423e-01,
        2.45410346e+01,  4.54852706e+01,  5.45643048e+01,  5.18066360e+01,
        1.30380487e+01,  3.36097259e+01,  4.92053513e+01,  7.58912851e+00,
        5.80806563e+00,  9.71533917e+00,  1.44386031e+01,  6.07059287e+00,
        4.71148370e+01,  2.19017989e+01,  2.22459431e+01,  6.70681072e+01,
        1.86419068e+01,  1.95608837e+01,  2.26965763e+01,  3.52204097e+01,
        2.84261613e+01,  5.04852817e+01,  2.01044235e+01,  9.06149233e+01,
        5.25883273e+01,  9.28995301e+01,  3.52462800e+01,  8.34712468e+01,
        4.03083564e+01,  4.23768474e+01,  3.76023359e+01,  5.60595822e+01,
        2.99277740e+01, -6.54457615e+00,  4.77389349e+01,  2.91015076e+01,
        6.88128315e+01,  1.72385879e+02,  1.27115820e+02,  1.07388713e+02,
        1.07427000e+02,  1.04138669e+02,  1.36370076e+01,  7.88109243e+01,
        2.78740557e+01,  4.77511188e+01,  2.96667444e+01,  7.33797128e+01,
        1.08529000e+01,  1.08099805e+01,  2.69003683e+01,  2.93009440e+01,
        3.22138508e+01,  1.24188228e+02,  1.30771747e+02,  8.62688605e+01,
        6.61206651e+01,  1.08991168e+02,  3.78545007e+01,  2.06349814e+02,
        3.04423207e+01,  1.21619161e+02,  5.37443607e+01,  0.00000000e+00,
        1.21960442e+02,  1.77486806e+02,  1.72421097e+02,  2.10088678e+02,
        2.04281270e+01,  2.63866384e+02,  6.28105572e+01,  4.72363781e+01,
        7.85767201e+01,  7.03285633e+01,  9.07808433e+01,  1.44289197e+02,
        6.07729778e+01,  9.40858653e+01,  1.49570188e+01,  4.83448764e+01,
        2.21138538e+01,  1.89770108e+01,  9.49136418e+00,  6.36937968e+01,
        5.60108519e+01,  5.63705626e+01,  5.02087073e+01,  5.66740673e+01,
        3.35614235e+01,  5.26988452e+01,  1.16337542e+02,  1.61726912e+02,
        1.00438367e+02,  8.70650882e+01,  7.20001183e+01,  7.55882574e+01,
        1.54747976e+02,  9.79604293e+01,  6.53819619e+01,  6.36794680e+01,
        2.16728886e+01,  4.48077096e+01,  1.58732731e+02,  2.31190597e+02,
        3.42908473e+01,  1.00659101e+02,  8.51281922e+01,  5.80533021e+00,
        9.49513872e+01,  2.93967465e+01,  1.14841360e+02,  6.18547000e+01,
        5.77493000e+01,  3.66476955e+01,  1.46558918e+01,  2.00406453e+01,
        1.54152458e+01,  7.08875071e+01,  3.06728867e+01,  4.10326461e+01,
        4.23564700e+01,  3.63109335e+01,  3.97477577e+01,  3.14334381e+01,
        5.61902272e+01,  3.81275179e+01,  1.61512768e+01,  6.67042826e+01,
        1.86974058e+01,  4.81415500e+01,  1.52662730e+01,  1.77161790e+02,
        2.39751877e+02,  1.44634235e+02,  2.65697669e+02,  3.03169013e+02,
        1.57387095e+02,  1.98151322e+02,  1.64601904e+02,  9.14829818e+01,
        8.87743633e+01,  8.99504461e+01,  8.87268082e+01,  8.88845554e+01,
        1.55426939e+02,  1.66448354e+02,  1.83866586e+02,  1.93302752e+02,
        2.01254751e+02,  1.42506657e+02,  9.98636739e+01,  7.09137771e+01,
        7.54556479e+01,  1.31864329e+02,  2.54619751e+02,  2.97574825e+02,
        2.76775823e+02,  3.00000915e+02,  2.19951209e+02,  2.60457482e+02,
        2.06290744e+02,  1.71733148e+02,  8.39542108e+01,  9.23861770e+01,
        2.22233618e+02,  1.41296535e+02,  1.69151129e+02,  2.28130641e+02,
        1.03337596e+02,  9.69696741e+01,  1.39068032e+02,  7.12952721e+01,
        1.38585790e+02,  1.60405681e+02,  1.49931661e+02,  2.28332482e+02,
        2.13861526e+02,  2.96146640e+02,  2.95746507e+02,  2.73025046e+02,
        2.08991808e+02,  2.78390613e+02,  1.37041745e+02,  9.10876042e+01,
        8.62907156e+01,  8.61757750e+01,  1.27812533e+02,  1.45138480e+02,
        1.48004582e+02,  2.29580263e+02,  1.74058567e+02,  1.71021063e+02,
        9.53415864e+01,  1.38105352e+02,  9.22994012e+01,  6.00939769e+01,
        7.08370286e+01,  1.35315510e+02,  1.81184490e+02,  2.20228099e+02,
        2.36697487e+02,  2.62610630e+02,  1.80695511e+02,  1.96925061e+02,
        2.04346478e+02,  3.88695080e+02,  1.38137667e+02,  1.24330945e+02,
        1.25954882e+02,  1.32934593e+02,  1.17891308e+02,  1.21167079e+02,
        1.16194343e+02,  1.43922881e+02,  1.48378823e+02,  1.34228266e+02,
        1.47447448e+02,  1.30660669e+02,  1.47108823e+02,  1.08291680e+02,
        7.16049331e+01,  1.18605075e+02,  5.41947288e+01,  9.38986731e+01,
        7.16746898e+01,  9.47368577e+01,  4.53174426e+01,  4.58977019e+01,
        9.45624534e+01,  3.77314040e+01,  3.37002121e+01,  2.55113135e+02,
        1.77040043e+02,  1.77097151e+02,  2.44208429e+02,  1.73038792e+02,
        1.23591236e+02,  1.15913834e+02,  1.88761015e+02,  1.73649019e+02,
        1.09967516e+02,  1.80646567e+02,  1.68224174e+02,  1.86834148e+02,
        1.76271768e+02,  1.23617422e+02,  1.06377795e+02,  1.35484945e+02,
        1.54756790e+02,  1.30447837e+02,  1.55231880e+02,  2.05557364e+02,
        1.10977771e+02,  1.64547768e+02,  1.32150326e+02,  1.01033156e+02,
        3.04644009e+02,  1.60279994e+02,  1.06210581e+02,  3.04903406e+02,
        1.15126634e+02,  9.66907013e+01,  6.67718689e+01,  3.93775127e+01,
        9.44185243e+01,  4.49422855e+01,  4.11823431e+01,  7.70771802e+01,
        4.39309254e+01,  2.92542458e+01,  5.17030969e+01,  8.87280894e+01,
        1.16889171e+02,  9.24289448e+01,  9.19408431e+01,  1.05609114e+02,
        7.99599297e+01,  1.78934110e+01,  8.71437384e+01,  5.51132793e+01,
        1.03029303e+02,  8.51912679e+01,  8.80096919e+01,  4.16630576e+01,
        1.62248571e+02,  8.34591844e+01,  1.03053003e+02,  5.18292352e+01,
        8.88893303e+00,  6.16404014e+01,  4.35953958e+01,  7.08183666e+00,
        6.18635999e+01,  5.04899972e+01,  4.57038470e+01,  8.96992080e+01,
        6.27565102e+01,  3.21701085e+01,  6.51572501e+01,  1.68161339e+01,
        1.03531403e+01,  1.04678419e+01,  5.11740889e+01,  3.14895301e+01,
        6.95456409e+01,  7.16518925e+01,  5.75139914e+01,  5.09124967e+01,
        6.28932421e+01,  2.96783242e+01, -2.00075192e-01,  2.30326847e+01,
        6.32846853e+01,  2.75192922e+01,  5.13591254e+01,  9.42552734e+01,
        3.48254998e+01,  4.82653308e+01,  4.57810267e+01,  6.65541253e+01,
        9.47081801e+01,  4.46982261e+01, -6.40039829e+00,  2.79780843e+01,
        2.62386468e+01,  2.74993617e+01,  3.11287288e+01,  7.05604622e+01,
        2.40440535e+02,  1.43551621e+02,  2.69154117e+01,  8.42736713e+01,
        1.02564232e+02,  8.05092834e+01,  7.92405907e+01,  4.91277859e+01,
        5.54558053e+01,  4.53217212e+01,  3.42925322e+01,  3.47017596e+01,
        2.86976433e+01,  7.42127578e+01,  1.15539232e+01,  4.16698003e+00,
        6.67607859e+01])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_8/test_residuals_histogram.png'
stats = OrderedDict([('R2', 0.7089583389998891), ('root_mean_squared_error', 49.59021474981029), ('mean_absolute_error', 32.86345940202088), ('rmse_over_stdev', 0.5145476787195726)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
